In [3]:
# This notebook contains the data pipeline to get from raw data to the inputs for the classifier, in order to train it to recognize 
# hand gestures.
# First, data is collected in the form of video taken mostly of myself (some of my mother), trying to use various capturing device and illumination 
# conditions, to improve the broadness of the training and test sets. Each video contains one gesture, in order to get labels without annotating.

# Raw videos are then processed frame by frame, extracting hand landmarkers using the mediapipe hand landmarker model.
# The obtained data is then stored in a csv file.


import mediapipe as mp
import cv2
import csv
import os
import numpy as np

mp_hands = mp.solutions.hands

# Main function for video processing

def video_to_data(video_path, filename):
    """Processes a video, extracts landmark data for both hands,
    and writes it to a CSV file.
    Inputs: - video path on the user device. - name of the output file.
    """
    #Checks the path exists
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"The video path {video_path} does not exist.")
    
    label = video_path[-5]  # Extract the video label from its filename ( the file name is in the format '...label1.mp4' .

    # We will use cv2 VideoCapture to handle videos frame by frame
    
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    # Main logic to extract data from each frame: it may be overkill to use every single frame, but it doesn't negatively impact the performance
    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5, max_num_hands=2) as hands: #Mediapipe model
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            # frame handling with some operations
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # flips the image horizontally
            image = cv2.flip(frame, 1)
            image.flags.writeable = False #improves performance
            results = hands.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # logic to append the results to the csv file
            if results.multi_hand_landmarks and results.multi_handedness: 
                for hand_landmarks, handedness_info in zip(results.multi_hand_landmarks, results.multi_handedness):
                    handedness = handedness_info.classification[0].label
                    handedness_num = 0 if handedness == "Right" else 1

                    x_coordinates = [landmark.x for landmark in hand_landmarks.landmark]
                    y_coordinates = [landmark.y for landmark in hand_landmarks.landmark]
                    z_coordinates = [landmark.z for landmark in hand_landmarks.landmark]

                    # final format of the data
                    features = np.array([handedness_num] + x_coordinates + y_coordinates + z_coordinates + [label], dtype=np.float32)

                    with open(filename, mode='a', newline='') as csvfile:
                        writer = csv.writer(csvfile)
                        writer.writerow(features.tolist())

            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()
    print("Finished processing video.")



In [4]:
#Repeats the last function on multiple files in a folder, to create a database
def videos_to_data(folder_path, output_csv):
    video_files = [f for f in os.listdir(folder_path) if f.endswith('.mp4')]  # List of all .mp4 videos
    
    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)
        video_to_data(video_path, output_csv)
    print("Finito")

In [7]:
#Used this to generate the database
videos_to_data('D:/progetto_video/videos', 'D:\progetto_video\dataset_nuovo_modello\dataset_new.csv')

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Marco\AppData\Local\Temp\ipykernel_8260\1019951029.py:2: SyntaxWarning: invalid escape sequence '\p'
  videos_to_data('D:/progetto_video/video_V', 'D:\progetto_video\dataset_nuovo_modello\dataset_new.csv')


Finished processing video.
Finished processing video.
Finito


In [9]:
import csv

# Sanity check

with open('D:\progetto_video\dataset_nuovo_modello\dataset.csv', 'r') as file:
    reader = csv.reader(file)
    lista = []
    a=0
    for row in reader:
        if a==0:
            print(len(row))
        a+=1


<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Marco\AppData\Local\Temp\ipykernel_8260\3108353051.py:2: SyntaxWarning: invalid escape sequence '\p'
  with open('D:\progetto_video\dataset_nuovo_modello\dataset.csv', 'r') as file:


65


In [2]:
# Example of some code used to engineer the dataset.

import csv

# Function to change all the examples sharing a label to another label
def update_csv(input_filename, output_filename, label1, label2):
    """
    Reads a CSV file, changes values in the last column from label1 to label2,
    and writes the updated data to a new CSV file.

    Args:
        input_filename: Path to the input CSV file.
        output_filename: Path to the output CSV file.
    """
    try:
        with open(input_filename, 'r', newline='', encoding='utf-8') as infile, \
                open(output_filename, 'w', newline='', encoding='utf-8') as outfile:

            reader = csv.reader(infile)
            writer = csv.writer(outfile)

            for row in reader:
                if row: #check if the row is not empty
                    if row[-1] == label1:  # Check the last element in the row
                        row[-1] = label2  # Modify the last element
                    writer.writerow(row)

    except FileNotFoundError:
        print(f"Error: Input file '{input_filename}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")



input_file = 'D:\progetto_video\dataset_nuovo_modello\dataset.csv'  
output_file = 'D:\progetto_video\dataset_nuovo_modello\dataset_new.csv' 

update_csv(input_file, output_file, "7.0", "8.0")
print(f"CSV file updated. Output written to '{output_file}'")



<>:32: SyntaxWarning: invalid escape sequence '\p'
<>:33: SyntaxWarning: invalid escape sequence '\p'
<>:32: SyntaxWarning: invalid escape sequence '\p'
<>:33: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Marco\AppData\Local\Temp\ipykernel_8260\1598850953.py:32: SyntaxWarning: invalid escape sequence '\p'
  input_file = 'D:\progetto_video\dataset_nuovo_modello\dataset.csv'  # Replace with your input file name
C:\Users\Marco\AppData\Local\Temp\ipykernel_8260\1598850953.py:33: SyntaxWarning: invalid escape sequence '\p'
  output_file = 'D:\progetto_video\dataset_nuovo_modello\dataset_new.csv' # Replace with your desired output file name


CSV file updated. Output written to 'D:\progetto_video\dataset_nuovo_modello\dataset_new.csv'
